### https://github.com/pyvista/pyvista/discussions/2567

In [1]:
import numpy as np
import xarray as xr
import pyvista as pv
import matplotlib

In [2]:
import ipywidgets as widgets
from ipywidgets import Button, HBox, VBox

In [3]:
ds = xr.open_dataset("https://thredds-su.ipsl.fr/thredds/dodsC/ipsl_thredds/brocksce/ICO/ICO.79.1jour.native.1_19790101_19790101_1D_inca_ges.nc")
ds

<xarray.Dataset>
Dimensions:               (cell: 16002, nvertex: 6, presnivs: 79,
                           time_counter: 24, axis_nbounds: 2)
Coordinates:
    lat                   (cell) float32 ...
    lon                   (cell) float32 ...
  * presnivs              (presnivs) float32 1.012e+05 1.009e+05 ... 5.297 1.483
    time_centered         (time_counter) datetime64[ns] ...
  * time_counter          (time_counter) datetime64[ns] 1979-01-01T00:30:00 ....
    time_instant          (time_counter) datetime64[ns] ...
Dimensions without coordinates: cell, nvertex, axis_nbounds
Data variables: (12/29)
    bounds_lon            (cell, nvertex) float32 ...
    bounds_lat            (cell, nvertex) float32 ...
    time_centered_bounds  (time_counter, axis_nbounds) datetime64[ns] ...
    time_counter_bounds   (time_counter, axis_nbounds) datetime64[ns] ...
    time_instant_bounds   (time_counter, axis_nbounds) datetime64[ns] ...
    pmid                  (time_counter, presnivs, cell) float64 ...
    ...                    ...
    vmrmcf                (time_counter, presnivs, cell) float64 ...
    vmrbe7                (time_counter, presnivs, cell) float64 ...
    vmrn2o                (time_counter, presnivs, cell) float64 ...
    vmrch4                (time_counter, presnivs, cell) float64 ...
    vmrco                 (time_counter, presnivs, cell) float64 ...
    vmrco2bih             (time_counter, presnivs, cell) float64 ...
Attributes:
    _NCProperties:                   version=1|netcdflibversion=4.6.0|hdf5lib...
    name:                            inca1d_ges
    description:                     Created by xios
    title:                           Created by xios
    Conventions:                     CF-1.6
    timeStamp:                       2022-Apr-05 08:59:15 GMT
    uuid:                            8982bdb7-b650-45c4-896c-e228bec0cb5a
    history:                         Fri Apr 15 12:05:37 2022: ncatted -O -a ...
    NCO:                             4.6.9
    DODS_EXTRA.Unlimited_Dimension:  time_counter

In [4]:
blon = ds['bounds_lon'].to_numpy()
blat = ds['bounds_lat'].to_numpy()
nvertex = blon.shape[-1]
print(nvertex)

blon = blon.reshape(-1, nvertex)
blat = blat.reshape(-1, nvertex)
print(blon.shape)

arr = ds['bounds_lon'].to_numpy()
blon = arr.reshape(-1, arr.shape[-1])
arr = ds['bounds_lat'].to_numpy()
blat = arr.reshape(-1, arr.shape[-1])

6
(16002, 6)


In [5]:
deg2rad = np.pi/180.
x = np.cos(blat*deg2rad)*np.cos(blon*deg2rad)
y = np.cos(blat*deg2rad)*np.sin(blon*deg2rad)
z = np.sin(blat*deg2rad)

In [6]:
points = np.stack((x,y,z), axis=2).reshape(x.size, 3)
faces = np.arange(x.shape[0] * nvertex).reshape(x.shape[0], nvertex)
faces = np.insert(faces, 0, nvertex, axis=1)

In [7]:
mesh = pv.PolyData(points, faces)
mesh

PolyData,Information
N Cells,16002
N Points,96012
N Strips,0
X Bounds,"-1.000e+00, 1.000e+00"
Y Bounds,"-9.998e-01, 9.998e-01"
Z Bounds,"-9.999e-01, 9.999e-01"
N Arrays,0


In [8]:
mesh = pv.PolyData(points, faces)
mesh.clean(inplace=True, tolerance=0.001)
mesh

PolyData,Information
N Cells,16002
N Points,32000
N Strips,0
X Bounds,"-1.000e+00, 1.000e+00"
Y Bounds,"-9.998e-01, 9.998e-01"
Z Bounds,"-9.999e-01, 9.999e-01"
N Arrays,0


In [9]:
var = ds['temp']
print(var.shape)

(24, 79, 16002)


In [10]:
kmax = var.shape[1]-1
lmax = var.shape[0]-1
kmax, lmax

(78, 23)

In [11]:
var = ds['temp'][0,0]
varmin = np.min(var)
varmax = np.max(var)
print("%f %f" %(varmin, varmax))

233.364045 302.943871


In [12]:
! wget -nc https://thredds-su.ipsl.fr/thredds/fileServer/ipsl_thredds/brocksce/pyvista/continents.vtk

Fichier «continents.vtk» déjà présent ; pas de récupération.



In [13]:
import pyvista as pv
from ipywidgets import interact

pv.set_jupyter_backend('trame')

pl = pv.Plotter()
pl.set_background('lightgray')

variable = 'temp'
kindex = 0
lindex = 0
var = ds[variable][lindex,kindex]
clim = [240, 300]
cmap = 'plasma'
show_edges = False

continents = pv.PolyData('continents.vtk')
pl.add_mesh(continents, color='white')
mesh_actor = pl.add_mesh(mesh, scalars=var, show_edges=show_edges, clim=clim, cmap=cmap, show_scalar_bar=False)
scalarBar_actor = pl.add_scalar_bar(mapper=mesh_actor.mapper, title=variable, color='black', fmt="%10.5f")    # fmt does nothing
pl.show(jupyter_backend='client')
pl.render()                              # why needed (if not the scene is darker)

def plot():
    global mesh_actor, scalarBar_actor
    # How to replace the 4 following lines; remove/add again might not be the most efficient way to update actors
    pl.remove_actor(mesh_actor)
    pl.remove_actor(scalarBar_actor)
    mesh_actor = pl.add_mesh(mesh, scalars=var, show_edges=show_edges, clim=clim, cmap=cmap, show_scalar_bar=False)
    scalarBar_actor = pl.add_scalar_bar(title=variable, color='black')
    pl.update()

widget_edges = widgets.Checkbox(value=show_edges, description='Edges')
widget_k = widgets.IntSlider(min=0, max=kmax, step=1, description='k-index:')
widget_l = widgets.IntSlider(min=0, max=lmax, step=1, description='l-index:')
widget_clim = widgets.Button(description="Update range")
widget_cmap = widgets.Dropdown(options=matplotlib.colormaps(), 
                               value=cmap, description='cmap:')
widget_variable = widgets.Dropdown(options=ds.data_vars.keys(), 
                                   value=variable, description='variable:')
#widget_output = widgets.Output()

def on_edges_change(change):
    global show_edges
    show_edges = change['new']
    plot()
    
def on_kvalue_change(change):
    global kindex
    kindex = change['new']
    update_variable()
    plot()

def on_lvalue_change(change):
    global lindex
    lindex = change['new']
    update_variable()
    plot()
    
def on_cmap_change(change):
    global cmap
    cmap = change['new']
    update_variable()
    plot()
    
def update_variable():
    global var
    lenVar = len(ds[variable].dims)
    if (lenVar == 1):
        var = ds[variable]
    elif (lenVar == 2):
        var = ds[variable][lindex]
    elif (lenVar == 3):
        var = ds[variable][lindex, kindex]
    
def on_variable_change(change):
    global variable
    variable = change['new']
    lenVar = len(ds[variable].dims)
    if (lenVar == 1):
        widget_k.disabled = True
        widget_l.disabled = True
        widget_k.value = 0
        widget_l.value = 0
    elif (lenVar == 2):
        widget_k.disabled = True
        widget_l.disabled = False
        widget_k.value = 0
    elif (lenVar == 3):
        widget_k.disabled = False
        widget_l.disabled = False
    update_variable()
    on_button_clicked(None)
    
def on_button_clicked(b):
    global clim
    varmin = float(np.min(var))
    varmax = float(np.max(var))
    clim = [varmin, varmax]
    plot()

widget_edges.observe(on_edges_change, names='value')
widget_k.observe(on_kvalue_change, names='value')
widget_l.observe(on_lvalue_change, names='value')
widget_cmap.observe(on_cmap_change, names='value')
widget_variable.observe(on_variable_change, names='value')
widget_clim.on_click(on_button_clicked)

display( widgets.HBox([widgets.VBox([widget_variable, widget_cmap]),
                       widgets.VBox([widget_k, widget_l]),
                       #widgets.VBox([widget_output]),
                       widgets.VBox([widget_clim, widget_edges])]) )

Widget(value="<iframe src='http://localhost:33265/index.html?ui=P_0x7fa74141dc30_0&reconnect=auto' style='widt…

In [15]:
print(pv.Report())


--------------------------------------------------------------------------------
  Date: Thu Feb 16 18:00:43 2023 CET

                OS : Linux
            CPU(s) : 8
           Machine : x86_64
      Architecture : 64bit
               RAM : 7.5 GiB
       Environment : Jupyter
       File system : ext4
        GPU Vendor : Intel Open Source Technology Center
      GPU Renderer : Mesa DRI Intel(R) UHD Graphics 620 (KBL GT2)
       GPU Version : 4.6 (Core Profile) Mesa 20.0.8

  Python 3.10.9 | packaged by conda-forge | (main, Feb  2 2023, 20:20:04) [GCC
  11.3.0]

           pyvista : 0.38.1
               vtk : 9.2.5
             numpy : 1.23.5
           imageio : 2.25.1
            scooby : 0.7.1
             pooch : v1.6.0
        matplotlib : 3.6.3
             PyQt5 : 5.15.7
           IPython : 8.10.0
             scipy : 1.10.0
              tqdm : 4.64.1
        jupyterlab : 3.6.1
         pythreejs : 2.4.1
             trame : 2.3.1
      trame_client : 2.6.1
      trame_